In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [4]:
col = pd.read_csv("../data/d10800_BS2_JoinedTrans.csv")

In [5]:
chunksize = 500
sample_prob = 0.05

In [6]:
df = pd.DataFrame(columns=col.columns)

In [129]:
def read_chunk(datapath, chunksize, sample_prob, columns):
    dataframe = pd.DataFrame()
    for chunk in pd.read_csv(datapath, chunksize=chunksize, usecols=columns):
        for index, row in chunk.iterrows():
            if random.uniform(0, 1) < sample_prob:
                dataframe = pd.concat([dataframe, pd.DataFrame([row])], ignore_index=True)
    return dataframe

In [7]:
for chunk in pd.read_csv("../data/d10800_10x_JoinedTrans.csv", chunksize=chunksize):
    for index, row in chunk.iterrows():
        if random.uniform(0, 1) < sample_prob:
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

In [8]:
df.head()

,src_file,row_id,hg38_ASF1A,hg38_CCT7,hg38_EPOR,hg38_NECAP2,hg38_LRRC8A,hg38_NDUFS8,hg38_MRPL22,hg38_HIST1H2BC,...,hg38_TBC1D10C,hg38_TMEM128,hg38_ASPSCR1,hg38_CDC20,hg38_DNAJB9,hg38_OSTM1,hg38_LCNL1,hg38_PHB,hg38_LRRC29,hg38_ZFP36
0,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,45,0,3,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,123,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,127,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,158,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,168,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [9]:
for chunk in pd.read_csv("../data/d10800_BS1_JoinedTrans.csv", chunksize=chunksize):
    for index, row in chunk.iterrows():
        if random.uniform(0, 1) < sample_prob:
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

In [10]:
for chunk in pd.read_csv("../data/d10800_BS2_JoinedTrans.csv", chunksize=chunksize):
    for index, row in chunk.iterrows():
        if random.uniform(0, 1) < sample_prob:
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

In [11]:
for chunk in pd.read_csv("../data/d10800_GEO_JoinedTrans.csv", chunksize=chunksize):
    for index, row in chunk.iterrows():
        if random.uniform(0, 1) < sample_prob:
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6260 entries, 0 to 6259
Columns: 10802 entries, src_file to hg38_ZFP36
dtypes: object(10802)
memory usage: 515.9+ MB


In [15]:
X = df.drop(columns=['row_id', 'src_file'])

In [16]:
y = df['src_file']

In [33]:
df_varience = X.var()
df_varience = df_varience.sort_values()
df_varience = df_varience.iloc[::-1]

In [34]:
df_varience

hg38_MALAT1           3938.170419
hg38_EEF1A1            793.027705
hg38_B2M               547.351779
hg38_RPS27             532.543348
hg38_TMSB4X             300.80294
                         ...     
hg38_RP11-67K19.3             0.0
hg38_NUDT10                   0.0
hg38_RP11-63A1.1              0.0
hg38_RP11-538I12.2            0.0
hg38_TCTEX1D1                 0.0
Length: 10800, dtype: object

In [29]:
df_varience_percentage = df_varience / df_varience.sum()
df_varience_percentage = df_varience_percentage.sort_values()
df_varience_percentage = df_varience_percentage.iloc[::-1]
df_varience_percentage

hg38_MALAT1           0.241257
hg38_EEF1A1           0.048582
hg38_B2M              0.033531
hg38_RPS27            0.032624
hg38_TMSB4X           0.018428
                        ...   
hg38_RP11-67K19.3          0.0
hg38_NUDT10                0.0
hg38_RP11-63A1.1           0.0
hg38_RP11-538I12.2         0.0
hg38_TCTEX1D1              0.0
Length: 10800, dtype: object

In [70]:
sum_percentage = 0
columns_selected = []

In [71]:
for i in range(len(df_varience_percentage)):
    sum_percentage += df_varience_percentage[i]
    columns_selected.append(df_varience_percentage.index[i])
    if sum_percentage >= 0.90:
        break

sum_percentage, len(columns_selected)

C:\Users\stefa\AppData\Local\Temp\ipykernel_3576\2572664672.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sum_percentage += df_varience_percentage[i]


(0.9001906016909633, 176)

In [78]:
X = X[columns_selected]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6260 entries, 0 to 6259
Columns: 176 entries, hg38_MALAT1 to hg38_CD79A
dtypes: object(176)
memory usage: 8.4+ MB


In [80]:
y.value_counts()

src_file
10x__TC__1-T22-OK-T8naive-10xG_027-CD8+_CD45RA+_Naive_Cytotoxic_T_qc    602
10x__TC__1-T11-OK-T4h-10xG_026-CD4+_Helper_T_qc                         529
10x__TC__1-T13-OK-T4naive-10xG_025-CD4+_CD45RA+_CD25-_Naive_T_qc        521
10x__TC__1-T21-OK-T8-10xG_022-CD8+_Cytotoxic_T_qc                       499
10x__TC__1-T12-OK-T4mem-10xG_023-CD4+_CD45RO+-_Memory_T_qc              478
                                                                       ... 
BS1__DC__1-OK-DC_cells_ALL-580-BOS-BROADS1-CHECKED_VB                     1
BS1__DC__1-OK-DC_cells_ALL-860-BOS-BROADS1-CHECKED_VB                     1
BS2__DC__DC_cells_pDC_V2_Arep2-BROADS2-CHECKED-VB                         1
BS2__DC__DC_cells_DC_V2_Arep2-BROADS2-CHECKED-VB                          1
BS1__TC__1-OK-T_cells_T4naive-860-NYC-BROADS1-CHECKED_VB                  1
Name: count, Length: 110, dtype: int64

In [82]:
from sklearn.feature_selection import RFECV 
from sklearn.ensemble import RandomForestClassifier

In [83]:
classifier = RandomForestClassifier()
recursive_selector = RFECV(classifier)

In [84]:
feat_select_model = recursive_selector.fit(X, y)

c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [85]:
feat_select_model.n_features_

93

In [103]:
feat_select_model.get_feature_names_out().shape

(93,)

In [86]:
X = feat_select_model.transform(X)

In [113]:
df = pd.DataFrame(data=X, columns=feat_select_model.get_feature_names_out())
df.head()

,hg38_MALAT1,hg38_EEF1A1,hg38_B2M,hg38_RPS27,hg38_TMSB4X,hg38_RPLP1,hg38_RPL10,hg38_RPL13,hg38_RPS12,hg38_RPS18,...,hg38_SH3BGRL3,hg38_EEF2,hg38_H3F3B,hg38_S100A6,hg38_VIM,hg38_UBC,hg38_LDHB,hg38_TYROBP,hg38_KLRB1,hg38_CD79A
0,64,6,10,23,14,15,28,29,10,27,...,2,4,4,0,1,4,2,0,0,3
1,45,15,10,28,10,21,29,32,21,21,...,0,1,5,0,0,4,0,0,0,6
2,33,5,7,9,9,14,9,20,13,14,...,0,0,1,0,0,4,0,0,0,3
3,52,13,33,20,23,26,26,39,15,15,...,4,2,4,1,1,5,1,0,0,2
4,28,6,5,15,10,12,23,32,14,25,...,0,1,4,0,0,0,2,0,0,5


In [114]:
df['src_file'] = y
df.head()

,hg38_MALAT1,hg38_EEF1A1,hg38_B2M,hg38_RPS27,hg38_TMSB4X,hg38_RPLP1,hg38_RPL10,hg38_RPL13,hg38_RPS12,hg38_RPS18,...,hg38_EEF2,hg38_H3F3B,hg38_S100A6,hg38_VIM,hg38_UBC,hg38_LDHB,hg38_TYROBP,hg38_KLRB1,hg38_CD79A,src_file
0,64,6,10,23,14,15,28,29,10,27,...,4,4,0,1,4,2,0,0,3,10x__BC__1-B01-OK-10xG_021-CD19+B_qc
1,45,15,10,28,10,21,29,32,21,21,...,1,5,0,0,4,0,0,0,6,10x__BC__1-B01-OK-10xG_021-CD19+B_qc
2,33,5,7,9,9,14,9,20,13,14,...,0,1,0,0,4,0,0,0,3,10x__BC__1-B01-OK-10xG_021-CD19+B_qc
3,52,13,33,20,23,26,26,39,15,15,...,2,4,1,1,5,1,0,0,2,10x__BC__1-B01-OK-10xG_021-CD19+B_qc
4,28,6,5,15,10,12,23,32,14,25,...,1,4,0,0,0,2,0,0,5,10x__BC__1-B01-OK-10xG_021-CD19+B_qc


In [123]:
corr = df.drop(columns=['src_file']).corr()
corr.mean().sort_values()

hg38_TYROBP    0.019834
hg38_GNLY      0.027979
hg38_NKG7      0.030473
hg38_JUNB      0.051547
hg38_KLRB1     0.140234
                 ...   
hg38_RPS27A    0.581626
hg38_RPS28     0.581813
hg38_RPS18     0.585559
hg38_RPS15A    0.586802
hg38_RPLP1     0.595019
Length: 93, dtype: float64

In [125]:
columns_selected = df.columns

In [130]:
df = pd.DataFrame()
df = pd.concat([df, read_chunk(datapath="../data/d10800_10x_JoinedTrans.csv", chunksize=chunksize, sample_prob=1.0, columns=columns_selected)], ignore_index=True)
df = pd.concat([df, read_chunk(datapath="../data/d10800_BS1_JoinedTrans.csv", chunksize=chunksize, sample_prob=1.0, columns=columns_selected)], ignore_index=True)
df = pd.concat([df, read_chunk(datapath="../data/d10800_BS2_JoinedTrans.csv", chunksize=chunksize, sample_prob=1.0, columns=columns_selected)], ignore_index=True)
df = pd.concat([df, read_chunk(datapath="../data/d10800_GEO_JoinedTrans.csv", chunksize=chunksize, sample_prob=1.0, columns=columns_selected)], ignore_index=True)

In [134]:
df['src_file'] = y
df.head()

,src_file,hg38_RPS19,hg38_RPS28,hg38_HLA-B,hg38_EIF1,hg38_LTB,hg38_RPS17,hg38_RPS5,hg38_MT-ND4,hg38_MT-CYB,...,hg38_RPL37A,hg38_RPL18A,hg38_RPL36,hg38_ACTB,hg38_HLA-C,hg38_RPL23A,hg38_GAPDH,hg38_CD79A,hg38_S100A6,hg38_RPL31
0,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,28,5,14,7,13,0,13,3,2,...,9,31,6,22,9,21,1,4,0,16
1,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,19,6,13,4,6,0,9,1,7,...,10,20,7,6,4,11,1,9,2,15
2,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,18,3,12,0,6,0,11,5,6,...,3,17,6,10,4,14,1,1,0,13
3,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,15,2,4,3,6,0,13,2,4,...,3,14,6,1,2,7,2,6,0,6
4,10x__BC__1-B01-OK-10xG_021-CD19+B_qc,27,4,1,2,6,0,10,2,6,...,6,26,6,9,2,18,0,6,0,6


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124162 entries, 0 to 124161
Data columns (total 94 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   src_file       124162 non-null  object
 1   hg38_RPS19     124162 non-null  int64 
 2   hg38_RPS28     124162 non-null  int64 
 3   hg38_HLA-B     124162 non-null  int64 
 4   hg38_EIF1      124162 non-null  int64 
 5   hg38_LTB       124162 non-null  int64 
 6   hg38_RPS17     124162 non-null  int64 
 7   hg38_RPS5      124162 non-null  int64 
 8   hg38_MT-ND4    124162 non-null  int64 
 9   hg38_MT-CYB    124162 non-null  int64 
 10  hg38_RPL27A    124162 non-null  int64 
 11  hg38_HLA-DRA   124162 non-null  int64 
 12  hg38_RPS26     124162 non-null  int64 
 13  hg38_SH3BGRL3  124162 non-null  int64 
 14  hg38_RPS18     124162 non-null  int64 
 15  hg38_EEF2      124162 non-null  int64 
 16  hg38_RPS3      124162 non-null  int64 
 17  hg38_IL32      124162 non-null  int64 
 18  hg38

In [141]:
from sklearn.model_selection import train_test_split

X = df.copy().drop(columns=['src_file'])
y = df['src_file']
X.head()

,hg38_RPS19,hg38_RPS28,hg38_HLA-B,hg38_EIF1,hg38_LTB,hg38_RPS17,hg38_RPS5,hg38_MT-ND4,hg38_MT-CYB,hg38_RPL27A,...,hg38_RPL37A,hg38_RPL18A,hg38_RPL36,hg38_ACTB,hg38_HLA-C,hg38_RPL23A,hg38_GAPDH,hg38_CD79A,hg38_S100A6,hg38_RPL31
0,28,5,14,7,13,0,13,3,2,11,...,9,31,6,22,9,21,1,4,0,16
1,19,6,13,4,6,0,9,1,7,20,...,10,20,7,6,4,11,1,9,2,15
2,18,3,12,0,6,0,11,5,6,16,...,3,17,6,10,4,14,1,1,0,13
3,15,2,4,3,6,0,13,2,4,7,...,3,14,6,1,2,7,2,6,0,6
4,27,4,1,2,6,0,10,2,6,11,...,6,26,6,9,2,18,0,6,0,6


In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [144]:
from joblib import dump
dump((X_train, X_test, y_train, y_test), 'preprocessed_data.joblib')

['preprocessed_data.joblib']